# Amazon Reviews Classification Modelling using Logistic Regression and XGBoost
```
@inproceedings{marc_reviews,
    title={The Multilingual Amazon Reviews Corpus},
    author={Keung, Phillip and Lu, Yichao and Szarvas, György and Smith, Noah A.},
    booktitle={Proceedings of the 2020 Conference on Empirical Methods in Natural Language Processing},
    year={2020}
}
```

In [2]:
import sys
from scipy.stats import uniform, randint

In [17]:
sys.path.append("/Users/dqmis/github/nlp-classification/")

from src.review.data.dataset import load_dataset
from src.review.models import baseline, ml_classifier
from src.review.data.dataset import split_dataset
from src.review import utils

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
dataset_df = load_dataset(return_pandas=True, languages=["en"], use_stars=False)

Found cached dataset amazon_reviews_multi (/Users/dqmis/.cache/huggingface/datasets/amazon_reviews_multi/default-18df3f9c3df27db5/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


In [6]:
dataset_df

,review_body,language,label
0,Arrived broken. Manufacturer defect. Two of th...,en,0.0
1,the cabinet dot were all detached from backing...,en,0.0
2,I received my first order of this product and ...,en,0.0
3,This product is a piece of shit. Do not buy. D...,en,0.0
4,went through 3 in one day doesn't fit correct ...,en,0.0
...,...,...,...
199995,"Cute slippers, my MIL loved them.",en,1.0
199996,My 6 year old likes this and keeps him engaged...,en,1.0
199997,Replaced my battery with it. Works like new.,en,1.0
199998,"I like them, holding up well.",en,1.0


In [7]:
# Split into train and test
train_df, val_df, test_df = split_dataset(dataset_df)

In [8]:
train_df

,review_body,language,label
17047,I can’t go a song without my phone saying the ...,en,0.0
159950,"I love that the box sorts 4 cash values, sorts...",en,1.0
23294,Had to return for refund minus shipping cost. ...,en,0.0
135704,I love the product material is strong and dura...,en,1.0
29929,Don't buy this product!!! After using these ca...,en,0.0
...,...,...,...
174028,I purchased this from Atoz merchant. At first ...,en,1.0
1885,Extremely cheap- worth $1-2 max. I will not pi...,en,0.0
60746,I love this blanket. Only problem is I was shi...,en,0.0
44421,These lights are really great in terms of outp...,en,0.0


Let's think about metric and the baseline. For metric we will be using F1 score to evaluate our model. For baseline we will be classifying text using top 10 most frequent words in each category.

In [9]:
# Get predictions of a baseline model

true_values = test_df["label"].values
predictions = test_df["review_body"].apply(baseline.classify).values

In [10]:
test_df.groupby("label").count()

,review_body,language
label,,
0.0,15874,15874
1.0,16126,16126


In [15]:
x_train, y_train = train_df["review_body"].values, train_df["label"].values
x_val, y_val = val_df["review_body"].values, val_df["label"].values
x_test, y_test = test_df["review_body"].values, test_df["label"].values

In [11]:
# Calculate metrics

utils.evaluate_model(true_values, predictions)
for metric, value in utils.evaluate_model(true_values, predictions).items():
    print(f"{metric}: {value}")

f1: 0.6114161360403259
accuracy: 0.63315625
classification_report:               precision    recall  f1-score   support

         0.0       0.59      0.88      0.70     15874
         1.0       0.76      0.39      0.52     16126

    accuracy                           0.63     32000
   macro avg       0.68      0.64      0.61     32000
weighted avg       0.68      0.63      0.61     32000



Let's train and evaluate logistic regression model.

In [25]:
# Logreg model
model = ml_classifier.MlClassifier(classifier_name="logreg")
model.fit(x_train, y_train)

# Calculate metrics
y_pred = model.predict(x_test)

for metric, value in utils.evaluate_model(y_test, y_pred).items():
    print(f"{metric}: {value}")

model.get_feature_importance()

/Users/dqmis/github/nlp-classification/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


f1: 0.8575891938277138
accuracy: 0.85759375
classification_report:               precision    recall  f1-score   support

         0.0       0.85      0.86      0.86     15874
         1.0       0.86      0.86      0.86     16126

    accuracy                           0.86     32000
   macro avg       0.86      0.86      0.86     32000
weighted avg       0.86      0.86      0.86     32000



Explanation(estimator='LogisticRegression()', description="\nFeatures with largest coefficients.\nCaveats:\n1. Be careful with features which are not\n   independent - weights don't show their importance.\n2. If scale of input features is different then scale of coefficients\n   will also be different, making direct comparison between coefficient values\n   incorrect.\n3. Depending on regularization, rare features sometimes may have high\n   coefficients; this doesn't mean they contribute much to the\n   classification result for most examples.\n", error=None, method='linear model', is_regression=False, targets=[TargetExplanation(target=1.0, feature_weights=FeatureWeights(pos=[FeatureWeight(feature='complaints', weight=2.861338934693038, std=None, value=None), FeatureWeight(feature='complaint', weight=2.631261410570389, std=None, value=None), FeatureWeight(feature='darling', weight=2.5870799172335635, std=None, value=None)], neg=[FeatureWeight(feature='disappointing', weight=-2.7963011215787397, std=None, value=None), FeatureWeight(feature='worthless', weight=-2.685746292720523, std=None, value=None), FeatureWeight(feature='dissatisfied', weight=-2.655939255960703, std=None, value=None), FeatureWeight(feature='poorly', weight=-2.622980600100965, std=None, value=None), FeatureWeight(feature='ridiculous', weight=-2.585406126629369, std=None, value=None), FeatureWeight(feature='shame', weight=-2.5692941029507175, std=None, value=None), FeatureWeight(feature='deleted', weight=-2.5471347585336686, std=None, value=None)], pos_remaining=18852, neg_remaining=17418), proba=None, score=None, weighted_spans=None, heatmap=None)], feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)

In [26]:
# XGBoost model
model = ml_classifier.MlClassifier(classifier_name="xgboost")
search_params = {
    "n_estimators": range(8, 20),
    "max_depth": range(3, 15),
    "learning_rate": [.4, .45, .5, .55, .6],
    "colsample_bytree": [.6, .7, .8, .9, 1]
}

# Hyperparameter search
model.hyperparam_search(x_train, y_train, search_params, n_iter=10)
model.fit(x_train, y_train)

# Calculate metrics
y_pred = model.predict(x_test)

for metric, value in utils.evaluate_model(y_test, y_pred).items():
    print(f"{metric}: {value}")

model.get_feature_importance()

f1: 0.8100600416241354
accuracy: 0.81021875
classification_report:               precision    recall  f1-score   support

         0.0       0.79      0.85      0.82     15874
         1.0       0.84      0.78      0.80     16126

    accuracy                           0.81     32000
   macro avg       0.81      0.81      0.81     32000
weighted avg       0.81      0.81      0.81     32000



Weight,Feature
0.0287,return
0.0285,loves
0.0273,waste
0.0240,apart
0.0226,horrible
0.0214,exactly
0.0204,perfect
0.0171,cheap
0.0166,enjoyed
0.0158,easy


In [24]:
# NB model
model = ml_classifier.MlClassifier(classifier_name="nb")
model.fit(x_train, y_train)

# Calculate metrics
y_pred = model.predict(x_test)

for metric, value in utils.evaluate_model(y_test, y_pred).items():
    print(f"{metric}: {value}")

model.get_feature_importance()

f1: 0.8407200747630316
accuracy: 0.840875
classification_report:               precision    recall  f1-score   support

         0.0       0.86      0.82      0.84     15874
         1.0       0.83      0.87      0.85     16126

    accuracy                           0.84     32000
   macro avg       0.84      0.84      0.84     32000
weighted avg       0.84      0.84      0.84     32000



Explanation(estimator='BernoulliNB()', description=None, error='estimator BernoulliNB() is not supported', method=None, is_regression=False, targets=None, feature_importances=None, decision_tree=None, highlight_spaces=None, transition_features=None, image=None)